# 🏥 Health Inequality Analysis

## Problem Statement
Health outcomes vary significantly across countries and income groups.
Quantifying these disparities is essential for prioritizing healthcare
investment and policy intervention.

This analysis investigates the relationship between economic capacity,
health spending, and population health outcomes.

## Policy Relevance
- Supports equitable healthcare resource allocation
- Highlights regions at high health risk
- Informs long-term public health planning

## Target Variables
- Life expectancy
- Mortality indicators
- Health expenditure
- Income / economic indicators

## Scope & Limitations
- Cross-country analysis (macro-level)
- Does not capture within-country inequality
- Data reflects reported national averages


## 🟦 Phase 1 — Data Loading & Initial Validation (WHO GHE DALYs)

### Objective
Ingest health burden data from the WHO Global Health Estimates (GHE)
using the OData API and validate its schema and coverage.

This phase focuses strictly on:
- API-based ingestion
- Schema inspection
- Metric availability validation
- Saving raw, unmodified data

❌ No filtering  
❌ No aggregation  
❌ No interpretation  

All downstream analysis will depend on validated availability
of required dimensions (year, sex, age, cause).


In [2]:
from utils.path_setup import setup_project_path
PROJECT_ROOT = setup_project_path()

import requests
import pandas as pd
from pathlib import Path

from src.utils.logger import get_logger

logger = get_logger("domain2_phase1_health")

print("Project root:", PROJECT_ROOT)


Project root: d:\def_main\Code\MyProjects\eda-mlops-portfolio


In [1]:
WHO_GHE_APIS = {
    2000: "https://xmart-api-public.who.int/DEX_CMS/GHE_FULL?$format=json",
    2010: "https://xmart-api-public.who.int/DEX_CMS/GHE_FULL?$format=json",
    2021: "https://xmart-api-public.who.int/DEX_CMS/GHE_FULL?$format=json"
}

WHO_GHE_APIS


{2000: 'https://xmart-api-public.who.int/DEX_CMS/GHE_FULL?$format=json',
 2010: 'https://xmart-api-public.who.int/DEX_CMS/GHE_FULL?$format=json',
 2021: 'https://xmart-api-public.who.int/DEX_CMS/GHE_FULL?$format=json'}

In [10]:
import requests
import pandas as pd

HEADERS = {
    "User-Agent": "Mozilla/5.0 (EDA-MLOps-Portfolio)",
    "Accept": "application/json"
}

def fetch_who_payload(url):
    r = requests.get(url, headers=HEADERS, timeout=60)
    r.raise_for_status()
    return pd.DataFrame(r.json()["value"])


In [11]:
# WHO GHE API access is intentionally disabled due to consistent 400 responses.
# See markdown explanation below.
API_DISABLED = True


### ⚠️ WHO XMART API Limitation (Phase 1)

Direct programmatic access to the WHO XMART `GHE_FULL` endpoint
returns consistent `400 Bad Request` responses when called via
Python HTTP clients.

This issue persists across:
- Query parameter variations
- Header modifications
- Pagination attempts

Therefore:
- Phase 1 focuses on schema understanding and data planning
- Programmatic ingestion is deferred to static exports in Phase 2
- This preserves notebook reproducibility and execution stability
